### Import Module

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation, LSTM, Dropout, TimeDistributed, Flatten
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.utils import class_weight
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from collections import Counter

### Load Data

In [2]:
training_load = pd.read_csv('training_1961_2016.csv', delimiter=',')
testing_2017_load = pd.read_csv('testing_2017.csv', delimiter=',')
testing_2018_load = pd.read_csv('testing_2018.csv', delimiter=',')
testing_2019_load = pd.read_csv('testing_2019.csv', delimiter=',')

In [3]:
training_load.head(5)

,FirstYear_index,FirstYear_playerID,FirstYear_yearID,FirstYear_teamID,FirstYear_lgID,FirstYear_stint,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,...,SixthYear_HBP,SixthYear_SH,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age
0,41261,aaronha01,1961,ML1,NL,1.0,155.0,603.0,115.0,197.0,...,1.0,0.0,8.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,32.0
1,41959,aaronha01,1962,ML1,NL,1.0,156.0,592.0,127.0,191.0,...,0.0,0.0,6.0,11.0,Hank,Aaron,Henry Louis,180.0,72.0,33.0
2,42719,aaronha01,1963,ML1,NL,1.0,161.0,631.0,121.0,201.0,...,1.0,0.0,5.0,21.0,Hank,Aaron,Henry Louis,180.0,72.0,34.0
3,43471,aaronha01,1964,ML1,NL,1.0,145.0,570.0,103.0,187.0,...,2.0,0.0,3.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,35.0
4,44225,aaronha01,1965,ML1,NL,1.0,150.0,570.0,109.0,181.0,...,2.0,0.0,6.0,13.0,Hank,Aaron,Henry Louis,180.0,72.0,36.0


In [4]:
var_name = ['index', 'playerID', 'teamID', 'lgID', 'stint','nameFirst', 'nameLast', 'nameGiven']

First = []
for i in range(len(var_name)):
    First.append(f"FirstYear_{var_name[i]}")

Second = []
for i in range(len(var_name)):
    Second.append(f"SecondYear_{var_name[i]}")

Third = []
for i in range(len(var_name)):
    Third.append(f"ThirdYear_{var_name[i]}")
    
Fourth = []
for i in range(len(var_name)):
    Fourth.append(f"FourthYear_{var_name[i]}")
    
Fifth = []
for i in range(len(var_name)):
    Fifth.append(f"FifthYear_{var_name[i]}")
    
FileColumnName = First + Second + Third + Fourth + Fifth

In [5]:
x_train = training_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(training_load), 5, 21))
y_train = training_load['SixthYear_HR'].values
x_test_2017 = testing_2017_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2017_load), 5, 21))
y_test_2017 = testing_2017_load['SixthYear_HR'].values
x_test_2018 = testing_2018_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2018_load), 5, 21))
y_test_2018 = testing_2018_load['SixthYear_HR'].values
x_test_2019 = testing_2019_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2019_load), 5, 21))
y_test_2019 = testing_2019_load['SixthYear_HR'].values

In [6]:
x_train_2017 = np.concatenate((x_train, x_test_2017))
y_train_2017 = np.concatenate((y_train, y_test_2017))

In [7]:
testing_2018 = testing_2018_load.drop(FileColumnName, axis=1).copy()

### Models

In [8]:
model_A = Sequential()
model_A.add(LSTM(128, input_shape=(5, 21), return_sequences=True))
model_A.add(LSTM(128, return_sequences=True))
model_A.add(Dropout(rate=0.5))
model_A.add(Flatten())
model_A.add(Dense(1024, activation="relu"))
model_A.add(Dropout(rate=0.5))
model_A.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_A.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_A.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 128)            76800     
                                                                 
 lstm_1 (LSTM)               (None, 5, 128)            131584    
                                                                 
 dropout (Dropout)           (None, 5, 128)            0         
                                                                 
 flatten (Flatten)           (None, 640)               0         
                                                                 
 dense (Dense)               (None, 1024)              656384    
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

2024-03-08 13:54:01.646907: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:01.647876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:01.648321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [9]:
model_B = Sequential()
model_B.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_B.add(LSTM(16, return_sequences=True))
model_B.add(LSTM(8, return_sequences=True))
model_B.add(Dropout(rate=0.5))
model_B.add(Flatten())
model_B.add(Dense(512, activation="relu"))
model_B.add(Dropout(rate=0.5))
model_B.add(Dense(64, activation="relu"))
model_B.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_B.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_B.summary()

2024-03-08 13:54:01.843812: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:01.844728: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:01.845357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 5, 32)             6912      
                                                                 
 lstm_3 (LSTM)               (None, 5, 16)             3136      
                                                                 
 lstm_4 (LSTM)               (None, 5, 8)              800       
                                                                 
 dropout_2 (Dropout)         (None, 5, 8)              0         
                                                                 
 flatten_1 (Flatten)         (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 512)               20992     
                                                                 
 dropout_3 (Dropout)         (None, 512)              

2024-03-08 13:54:02.072179: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:02.072795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:02.073336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [10]:
model_C = Sequential()
model_C.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_C.add(LSTM(32, return_sequences=True))
model_C.add(BatchNormalization())
model_C.add(Flatten())
model_C.add(Dense(512, activation="relu"))
model_C.add(BatchNormalization())
model_C.add(Dense(64, activation="relu"))
model_C.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate= 0.001)
model_C.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_C.summary()

2024-03-08 13:54:02.193814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:02.194286: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:02.195315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 5, 32)             6912      
                                                                 
 lstm_6 (LSTM)               (None, 5, 32)             8320      
                                                                 
 batch_normalization (BatchN  (None, 5, 32)            128       
 ormalization)                                                   
                                                                 
 flatten_2 (Flatten)         (None, 160)               0         
                                                                 
 dense_5 (Dense)             (None, 512)               82432     
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

2024-03-08 13:54:02.279504: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:02.280180: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:02.280926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [11]:
model_D = Sequential()
model_D.add(LSTM(64, input_shape=(5, 21), return_sequences=True))
model_D.add(LSTM(64, return_sequences=True))
model_D.add(Dropout(rate=0.5))
model_D.add(TimeDistributed(Dense(10)))
model_D.add(Flatten())
model_D.add(Dense(512, activation="relu"))
model_D.add(Dropout(rate=0.5))
model_D.add(Dense(64, activation="relu"))
model_D.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_D.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_D.summary()

2024-03-08 13:54:02.413025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:02.413461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:02.414095: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 5, 64)             22016     
                                                                 
 lstm_8 (LSTM)               (None, 5, 64)             33024     
                                                                 
 dropout_4 (Dropout)         (None, 5, 64)             0         
                                                                 
 time_distributed (TimeDistr  (None, 5, 10)            650       
 ibuted)                                                         
                                                                 
 flatten_3 (Flatten)         (None, 50)                0         
                                                                 
 dense_9 (Dense)             (None, 512)               26112     
                                                      

2024-03-08 13:54:02.495638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:02.496650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:02.497157: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [12]:
model_E = Sequential()
model_E.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_E.add(LSTM(32, return_sequences=True))
model_E.add(Dropout(rate=0.5))
model_E.add(Flatten())
model_E.add(Dense(512, activation="relu"))
model_E.add(Dropout(rate=0.5))
model_E.add(Dense(64, activation="relu"))
model_E.add(Dense(1, activation='relu'))
adam = optimizers.Adam(learning_rate = 0.001)
model_E.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_E.summary()

2024-03-08 13:54:02.615389: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:02.616037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:02.616568: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 5, 32)             6912      
                                                                 
 lstm_10 (LSTM)              (None, 5, 32)             8320      
                                                                 
 dropout_6 (Dropout)         (None, 5, 32)             0         
                                                                 
 flatten_4 (Flatten)         (None, 160)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               82432     
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 64)               

### Training

In [13]:
from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2017),
                                                  y=y_train_2017)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2017), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "A/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_A_stat = model_A.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 13:54:02.766225: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-08 13:54:02.874364: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:02.875543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:02.876316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG IN

75/77 [============================>.] - ETA: 0s - loss: 517.3497 - accuracy: 0.0897

2024-03-08 13:54:05.547468: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 13:54:05.547995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 13:54:05.548537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.07065, saving model to A/saved-model-001-0.0707.hdf5
77/77 [==============================] - 3s 23ms/step - loss: 512.9644 - accuracy: 0.0896 - val_loss: 681.8196 - val_accuracy: 0.0707
Epoch 2/1000
76/77 [============================>.] - ETA: 0s - loss: 405.8748 - accuracy: 0.0895
Epoch 2: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 19ms/step - loss: 404.5285 - accuracy: 0.0899 - val_loss: 621.9222 - val_accuracy: 0.0707
Epoch 3/1000
75/77 [============================>.] - ETA: 0s - loss: 442.1033 - accuracy: 0.0905
Epoch 3: val_accuracy did not improve from 0.07065
77/77 [==============================] - 2s 20ms/step - loss: 437.3110 - accuracy: 0.0899 - val_loss: 437.3694 - val_accuracy: 0.0707
Epoch 4/1000
76/77 [============================>.] - ETA: 0s - loss: 394.2114 - accuracy: 0.0896
Epoch 4: val_accuracy did not improve from 0.07065
77/77 [==============================] - 2s 20ms/st

In [14]:
from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2017),
                                                  y=y_train_2017)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2017), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "B/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_B_stat = model_B.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 14:25:11.118465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 14:25:11.119094: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 14:25:11.119660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

74/77 [===========================>..] - ETA: 0s - loss: 878.2239 - accuracy: 0.0911

2024-03-08 14:25:13.641106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 14:25:13.641649: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 14:25:13.642236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.07065, saving model to B/saved-model-001-0.0707.hdf5
77/77 [==============================] - 3s 14ms/step - loss: 878.8493 - accuracy: 0.0908 - val_loss: 262.0995 - val_accuracy: 0.0707
Epoch 2/1000
71/77 [==========================>...] - ETA: 0s - loss: 453.5524 - accuracy: 0.0904
Epoch 2: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 8ms/step - loss: 466.2386 - accuracy: 0.0899 - val_loss: 405.5454 - val_accuracy: 0.0707
Epoch 3/1000
75/77 [============================>.] - ETA: 0s - loss: 395.6059 - accuracy: 0.0898
Epoch 3: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 8ms/step - loss: 392.8468 - accuracy: 0.0899 - val_loss: 217.1703 - val_accuracy: 0.0707
Epoch 4/1000
75/77 [============================>.] - ETA: 0s - loss: 416.8903 - accuracy: 0.0886
Epoch 4: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 7ms/step 

In [15]:
from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2017),
                                                  y=y_train_2017)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2017), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "C/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_C_stat = model_C.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 14:33:50.362517: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 14:33:50.363011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 14:33:50.363722: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

71/77 [==========================>...] - ETA: 0s - loss: 801.7144 - accuracy: 0.0915

2024-03-08 14:33:52.259484: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 14:33:52.259951: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 14:33:52.260648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.07065, saving model to C/saved-model-001-0.0707.hdf5
77/77 [==============================] - 2s 12ms/step - loss: 759.0015 - accuracy: 0.0914 - val_loss: 124.3181 - val_accuracy: 0.0707
Epoch 2/1000
75/77 [============================>.] - ETA: 0s - loss: 413.9296 - accuracy: 0.0968
Epoch 2: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 10ms/step - loss: 413.3931 - accuracy: 0.0966 - val_loss: 109.7628 - val_accuracy: 0.0707
Epoch 3/1000
77/77 [==============================] - ETA: 0s - loss: 356.1284 - accuracy: 0.0914
Epoch 3: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 11ms/step - loss: 356.1284 - accuracy: 0.0914 - val_loss: 83.4499 - val_accuracy: 0.0707
Epoch 4/1000
74/77 [===========================>..] - ETA: 0s - loss: 367.8813 - accuracy: 0.0970
Epoch 4: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 10ms/ste

In [16]:
""" from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2017)
                                               ,y_train_2017)

filepath = "D/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_D_stat = model_D.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             ) """

from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2017),
                                                  y=y_train_2017)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2017), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "D/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_D_stat = model_D.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 14:46:53.816954: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 14:46:53.817987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 14:46:53.818677: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

72/77 [===========================>..] - ETA: 0s - loss: 670.4314 - accuracy: 0.0900

2024-03-08 14:46:55.912904: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 14:46:55.913927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 14:46:55.914457: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.07065, saving model to D/saved-model-001-0.0707.hdf5
77/77 [==============================] - 2s 14ms/step - loss: 695.1332 - accuracy: 0.0905 - val_loss: 606.1126 - val_accuracy: 0.0707
Epoch 2/1000
76/77 [============================>.] - ETA: 0s - loss: 454.3414 - accuracy: 0.0903
Epoch 2: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 10ms/step - loss: 452.6486 - accuracy: 0.0899 - val_loss: 259.9753 - val_accuracy: 0.0707
Epoch 3/1000
77/77 [==============================] - ETA: 0s - loss: 426.1729 - accuracy: 0.0899
Epoch 3: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 10ms/step - loss: 426.1729 - accuracy: 0.0899 - val_loss: 288.1377 - val_accuracy: 0.0707
Epoch 4/1000
76/77 [============================>.] - ETA: 0s - loss: 411.5912 - accuracy: 0.0898
Epoch 4: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 10ms/st

In [17]:
from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2017),
                                                  y=y_train_2017)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2017), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "D/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_D_stat = model_D.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


75/77 [============================>.] - ETA: 0s - loss: 122.9414 - accuracy: 0.0897
Epoch 1: val_accuracy improved from -inf to 0.07065, saving model to D/saved-model-001-0.0707.hdf5
77/77 [==============================] - 1s 12ms/step - loss: 133.0277 - accuracy: 0.0896 - val_loss: 131.0010 - val_accuracy: 0.0707
Epoch 2/1000
76/77 [============================>.] - ETA: 0s - loss: 120.2796 - accuracy: 0.0898
Epoch 2: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 11ms/step - loss: 120.4845 - accuracy: 0.0898 - val_loss: 202.7153 - val_accuracy: 0.0707
Epoch 3/1000
75/77 [============================>.] - ETA: 0s - loss: 130.2466 - accuracy: 0.0901
Epoch 3: val_accuracy did not improve from 0.07065
77/77 [==============================] - 1s 13ms/step - loss: 128.9614 - accuracy: 0.0899 - val_loss: 133.6390 - val_accuracy: 0.0707
Epoch 4/1000
75/77 [============================>.] - ETA: 0s - loss: 135.6429 - accuracy: 0.0908
Epoch 4: val_accur

### Analysis

In [18]:
AAA = load_model('2018/saved-model-070-0.1196-A.hdf5')
BBB = load_model('2018/saved-model-107-0.1359-B.hdf5')
CCC = load_model('2018/saved-model-262-0.1087-C.hdf5')
DDD = load_model('2018/saved-model-988-0.1033-D.hdf5')
EEE = load_model('2018/saved-model-524-0.1250-E.hdf5')

In [19]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = AAA.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("Correct answer：%.4f" % (t/len(y_test_2018)))
print("plus or minus 1：%.4f" % (a/len(y_test_2018)))
print("plus or minus 3：%.4f" % (b/len(y_test_2018)))
print("plus or minus 5：%.4f" % (c/len(y_test_2018)))
print("plus or minu 10：%.4f" % (d/len(y_test_2018)))

6/6 [==============================] - 0s 3ms/step
Correct answer：0.1196
plus or minus 1：0.2174
plus or minus 3：0.4511
plus or minus 5：0.5815
plus or minu 10：0.8533


In [20]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.518795177178538
mean_squared_error: 55.11087596475249
rmse: 7.423669979515017


In [21]:
Counter(sorted(p))

Counter({4.0: 3,
         10.0: 3,
         2.0: 2,
         13.0: 2,
         16.0: 2,
         0.0: 1,
         1.0: 1,
         3.0: 1,
         8.0: 1,
         11.0: 1,
         14.0: 1,
         15.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1})

In [22]:
from collections import Counter

Counter(sorted(n))

Counter({10.0: 5,
         2.0: 4,
         8.0: 4,
         1.0: 3,
         4.0: 3,
         13.0: 3,
         0.0: 2,
         5.0: 2,
         9.0: 2,
         14.0: 2,
         16.0: 2,
         3.0: 1,
         7.0: 1,
         11.0: 1,
         12.0: 1,
         15.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1})

In [23]:
Counter(sorted(m))

Counter({1.0: 6,
         4.0: 6,
         11.0: 6,
         13.0: 6,
         6.0: 5,
         8.0: 5,
         10.0: 5,
         14.0: 5,
         15.0: 5,
         0.0: 4,
         2.0: 4,
         5.0: 4,
         9.0: 4,
         3.0: 3,
         7.0: 3,
         12.0: 3,
         16.0: 3,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         21.0: 1,
         23.0: 1,
         25.0: 1})

In [24]:
Counter(sorted(q))

Counter({23.0: 4,
         27.0: 3,
         20.0: 2,
         21.0: 2,
         36.0: 2,
         37.0: 2,
         0.0: 1,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         22.0: 1,
         24.0: 1,
         28.0: 1,
         38.0: 1,
         39.0: 1,
         43.0: 1,
         48.0: 1})

In [25]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [4, 10, 15, 22, 45, 47, 51, 57, 70, 73, 74, 77, 94, 95, 103, 108, 129, 136, 152, 168, 176, 177]
1: [4, 5, 8, 10, 13, 15, 22, 28, 34, 45, 47, 48, 49, 51, 54, 57, 70, 73, 74, 77, 81, 85, 94, 95, 103, 108, 113, 114, 115, 129, 136, 151, 152, 154, 155, 160, 168, 176, 177, 182]
3: [2, 4, 5, 8, 10, 11, 13, 15, 16, 22, 28, 34, 39, 42, 45, 46, 47, 48, 49, 51, 53, 54, 57, 58, 59, 62, 65, 68, 69, 70, 71, 73, 74, 77, 78, 81, 83, 85, 86, 92, 93, 94, 95, 96, 98, 99, 101, 103, 105, 108, 113, 114, 115, 116, 117, 120, 121, 124, 125, 129, 132, 133, 136, 142, 144, 151, 152, 153, 154, 155, 156, 160, 161, 162, 166, 168, 169, 172, 176, 177, 180, 181, 182]
NA: [0, 3, 7, 9, 18, 19, 21, 25, 32, 35, 38, 41, 44, 52, 76, 82, 87, 104, 106, 110, 111, 122, 126, 135, 149, 167, 179]


In [26]:
AA = testing_2018.copy()
AA['Prediction'] = np.round(predictions)
AA['Difference'] = AA.SixthYear_HR - AA.Prediction

In [27]:
BB = testing_2018.copy()
BB['Prediction'] = np.round(predictions)
BB['Difference'] = BB.SixthYear_HR - BB.Prediction

In [28]:
CC = testing_2018.copy()
CC['Prediction'] = np.round(predictions)
CC['Difference'] = CC.SixthYear_HR - CC.Prediction

In [29]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = BBB.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2018)))
print("plus or minus 1：%.4f" % (a/len(y_test_2018)))
print("plus or minus 3：%.4f" % (b/len(y_test_2018)))
print("plus or minus 5：%.4f" % (c/len(y_test_2018)))
print("plus or minus 10：%.4f" % (d/len(y_test_2018)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1359
plus or minus 1：0.2391
plus or minus 3：0.4130
plus or minus 5：0.5652
plus or minus 10：0.8152


In [30]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.783073325519976
mean_squared_error: 59.662751619402094
rmse: 7.724166726540934


In [31]:
Counter(sorted(p))

Counter({4.0: 3,
         10.0: 3,
         14.0: 3,
         2.0: 2,
         3.0: 2,
         6.0: 2,
         9.0: 2,
         12.0: 2,
         16.0: 2,
         7.0: 1,
         13.0: 1,
         17.0: 1,
         21.0: 1})

In [32]:
Counter(sorted(n))

Counter({13.0: 5,
         14.0: 5,
         2.0: 3,
         3.0: 3,
         4.0: 3,
         9.0: 3,
         10.0: 3,
         15.0: 3,
         1.0: 2,
         6.0: 2,
         8.0: 2,
         12.0: 2,
         16.0: 2,
         5.0: 1,
         7.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         25.0: 1})

In [33]:
Counter(sorted(m))

Counter({11.0: 6,
         13.0: 6,
         1.0: 5,
         4.0: 5,
         8.0: 5,
         10.0: 5,
         14.0: 5,
         0.0: 4,
         5.0: 4,
         6.0: 4,
         9.0: 4,
         15.0: 4,
         2.0: 3,
         3.0: 3,
         12.0: 3,
         16.0: 3,
         7.0: 2,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         21.0: 1,
         25.0: 1})

In [34]:
Counter(sorted(q))

Counter({23.0: 5,
         27.0: 3,
         15.0: 2,
         20.0: 2,
         36.0: 2,
         37.0: 2,
         38.0: 2,
         1.0: 1,
         2.0: 1,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         22.0: 1,
         24.0: 1,
         25.0: 1,
         28.0: 1,
         30.0: 1,
         33.0: 1,
         34.0: 1,
         39.0: 1,
         43.0: 1,
         48.0: 1})

In [35]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [2, 10, 13, 28, 34, 45, 47, 62, 70, 94, 98, 103, 108, 113, 114, 117, 120, 121, 129, 151, 152, 168, 169, 173, 180]
1: [2, 4, 10, 11, 13, 15, 16, 28, 34, 42, 45, 46, 47, 48, 54, 57, 62, 70, 74, 85, 93, 94, 95, 98, 99, 103, 108, 113, 114, 115, 117, 120, 121, 124, 129, 151, 152, 153, 168, 169, 173, 177, 180, 182]
3: [1, 2, 4, 5, 8, 10, 11, 13, 15, 16, 22, 28, 34, 37, 39, 42, 45, 46, 47, 48, 49, 51, 53, 54, 57, 62, 65, 70, 73, 74, 77, 78, 79, 81, 83, 85, 86, 92, 93, 94, 95, 98, 99, 101, 103, 108, 113, 114, 115, 116, 117, 120, 121, 124, 125, 127, 128, 129, 142, 144, 151, 152, 153, 154, 155, 160, 162, 168, 169, 173, 176, 177, 178, 180, 181, 182]
NA: [3, 7, 9, 18, 19, 21, 25, 35, 36, 38, 41, 44, 52, 64, 66, 76, 80, 84, 87, 88, 104, 106, 110, 111, 122, 126, 135, 138, 149, 163, 165, 167, 170, 179]


In [36]:
DD = testing_2018.copy()
DD['Prediction'] = np.round(predictions)
DD['Difference'] = DD.SixthYear_HR - DD.Prediction

In [37]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = CCC.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2018)))
print("plus or minus 1：%.4f" % (a/len(y_test_2018)))
print("plus or minus 3：%.4f" % (b/len(y_test_2018)))
print("plus or minus 5：%.4f" % (c/len(y_test_2018)))
print("plus or minus 10：%.4f" % (d/len(y_test_2018)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1087
plus or minus 1：0.2174
plus or minus 3：0.4076
plus or minus 5：0.5870
plus or minus 10：0.8913


In [38]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.312735343594914
mean_squared_error: 47.678191052728224
rmse: 6.904939612533061


In [39]:
Counter(sorted(p))

Counter({0.0: 2,
         1.0: 2,
         5.0: 2,
         6.0: 2,
         8.0: 2,
         14.0: 2,
         2.0: 1,
         4.0: 1,
         7.0: 1,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         12.0: 1,
         18.0: 1})

In [40]:
Counter(sorted(n))

Counter({1.0: 5,
         8.0: 5,
         4.0: 4,
         0.0: 2,
         2.0: 2,
         3.0: 2,
         5.0: 2,
         6.0: 2,
         13.0: 2,
         14.0: 2,
         7.0: 1,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         12.0: 1,
         15.0: 1,
         18.0: 1,
         19.0: 1,
         20.0: 1,
         21.0: 1,
         23.0: 1,
         25.0: 1})

In [41]:
Counter(sorted(m))

Counter({1.0: 6,
         4.0: 6,
         6.0: 6,
         8.0: 5,
         0.0: 4,
         2.0: 4,
         5.0: 4,
         9.0: 4,
         10.0: 4,
         11.0: 4,
         13.0: 4,
         14.0: 4,
         3.0: 3,
         16.0: 3,
         12.0: 2,
         15.0: 2,
         23.0: 2,
         7.0: 1,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         25.0: 1})

In [42]:
Counter(sorted(q))

Counter({23.0: 4,
         20.0: 2,
         27.0: 2,
         36.0: 2,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         12.0: 1,
         21.0: 1,
         22.0: 1,
         37.0: 1,
         43.0: 1,
         48.0: 1})

In [43]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [8, 10, 15, 28, 48, 77, 78, 86, 90, 98, 115, 116, 132, 136, 155, 160, 169, 173, 176, 177]
1: [2, 5, 8, 10, 15, 16, 28, 39, 48, 49, 51, 54, 58, 77, 78, 81, 85, 86, 90, 94, 98, 115, 116, 119, 124, 132, 136, 144, 150, 151, 152, 153, 155, 156, 158, 160, 169, 173, 176, 177]
3: [2, 5, 8, 10, 11, 13, 15, 16, 20, 22, 28, 34, 39, 42, 47, 48, 49, 51, 53, 54, 58, 62, 65, 69, 70, 71, 73, 74, 77, 78, 79, 81, 83, 85, 86, 90, 93, 94, 98, 99, 101, 103, 113, 114, 115, 116, 117, 119, 120, 124, 129, 131, 132, 136, 144, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 162, 168, 169, 171, 172, 173, 176, 177, 180, 182]
NA: [9, 14, 18, 19, 21, 25, 35, 38, 41, 44, 52, 76, 87, 104, 110, 111, 122, 140, 175, 179]


In [44]:
EE = testing_2018.copy()
EE['Prediction'] = np.round(predictions)
EE['Difference'] = EE.SixthYear_HR - EE.Prediction

In [45]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = DDD.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2018)))
print("plus or minus 1：%.4f" % (a/len(y_test_2018)))
print("plus or minus 3：%.4f" % (b/len(y_test_2018)))
print("plus or minus 5：%.4f" % (c/len(y_test_2018)))
print("plus or minus 10：%.4f" % (d/len(y_test_2018)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1033
plus or minus 1：0.2283
plus or minus 3：0.4565
plus or minus 5：0.5870
plus or minus 10：0.9022


In [46]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.20205302082974
mean_squared_error: 47.53985116889178
rmse: 6.894914877566784


In [47]:
Counter(sorted(p))

Counter({2.0: 3,
         8.0: 3,
         10.0: 3,
         4.0: 2,
         5.0: 1,
         6.0: 1,
         7.0: 1,
         11.0: 1,
         13.0: 1,
         14.0: 1,
         15.0: 1,
         21.0: 1})

In [48]:
Counter(sorted(n))

Counter({1.0: 5,
         8.0: 4,
         10.0: 4,
         2.0: 3,
         3.0: 3,
         6.0: 3,
         4.0: 2,
         5.0: 2,
         11.0: 2,
         12.0: 2,
         13.0: 2,
         14.0: 2,
         15.0: 2,
         7.0: 1,
         9.0: 1,
         17.0: 1,
         18.0: 1,
         20.0: 1,
         21.0: 1})

In [49]:
Counter(sorted(m))

Counter({1.0: 8,
         4.0: 7,
         6.0: 6,
         14.0: 6,
         2.0: 5,
         8.0: 5,
         10.0: 5,
         11.0: 5,
         15.0: 5,
         0.0: 4,
         5.0: 4,
         3.0: 3,
         9.0: 3,
         12.0: 3,
         13.0: 3,
         18.0: 2,
         21.0: 2,
         7.0: 1,
         16.0: 1,
         17.0: 1,
         19.0: 1,
         20.0: 1,
         22.0: 1,
         23.0: 1,
         32.0: 1})

In [50]:
Counter(sorted(q))

Counter({20.0: 2,
         23.0: 2,
         27.0: 2,
         36.0: 2,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         11.0: 1,
         13.0: 1,
         24.0: 1,
         38.0: 1,
         43.0: 1,
         48.0: 1})

In [51]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [5, 8, 10, 13, 16, 22, 47, 48, 54, 81, 85, 98, 113, 116, 125, 150, 158, 160, 176]
1: [2, 5, 8, 10, 13, 15, 16, 20, 22, 28, 34, 39, 47, 48, 49, 53, 54, 74, 78, 81, 85, 86, 90, 93, 94, 98, 108, 113, 115, 116, 119, 124, 125, 132, 150, 156, 158, 160, 166, 168, 173, 176]
3: [2, 5, 8, 10, 11, 13, 15, 16, 20, 22, 28, 34, 39, 46, 47, 48, 49, 50, 51, 53, 54, 55, 57, 62, 65, 69, 73, 74, 77, 78, 79, 81, 83, 85, 86, 90, 92, 93, 94, 95, 98, 99, 101, 103, 108, 113, 114, 115, 116, 117, 119, 120, 123, 124, 125, 129, 131, 132, 134, 136, 137, 142, 144, 150, 151, 152, 154, 155, 156, 158, 159, 160, 161, 162, 166, 168, 169, 171, 173, 174, 176, 177, 180, 182]
NA: [9, 18, 19, 21, 25, 38, 40, 44, 75, 76, 87, 104, 110, 111, 122, 128, 165, 179]


In [52]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = EEE.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2018)))
print("plus or minus 1：%.4f" % (a/len(y_test_2018)))
print("plus or minus 3：%.4f" % (b/len(y_test_2018)))
print("plus or minus 5：%.4f" % (c/len(y_test_2018)))
print("plus or minus 10：%.4f" % (d/len(y_test_2018)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1250
plus or minus 1：0.2446
plus or minus 3：0.4674
plus or minus 5：0.6087
plus or minus 10：0.8750


In [53]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.17603129018908
mean_squared_error: 47.71779569843006
rmse: 6.907806866034259


In [54]:
Counter(sorted(p))

Counter({8.0: 3,
         10.0: 3,
         5.0: 2,
         11.0: 2,
         12.0: 2,
         14.0: 2,
         21.0: 2,
         3.0: 1,
         7.0: 1,
         9.0: 1,
         15.0: 1,
         18.0: 1,
         19.0: 1,
         25.0: 1})

In [55]:
Counter(sorted(n))

Counter({4.0: 5,
         14.0: 5,
         8.0: 4,
         6.0: 3,
         9.0: 3,
         10.0: 3,
         11.0: 3,
         12.0: 3,
         2.0: 2,
         5.0: 2,
         15.0: 2,
         21.0: 2,
         3.0: 1,
         7.0: 1,
         13.0: 1,
         18.0: 1,
         19.0: 1,
         22.0: 1,
         23.0: 1,
         25.0: 1})

In [56]:
Counter(sorted(m))

Counter({11.0: 8,
         9.0: 7,
         4.0: 6,
         1.0: 5,
         6.0: 5,
         10.0: 5,
         13.0: 5,
         14.0: 5,
         15.0: 5,
         5.0: 4,
         8.0: 4,
         0.0: 3,
         2.0: 3,
         3.0: 3,
         12.0: 3,
         20.0: 3,
         7.0: 2,
         16.0: 2,
         21.0: 2,
         23.0: 2,
         18.0: 1,
         19.0: 1,
         22.0: 1,
         25.0: 1})

In [57]:
Counter(sorted(q))

Counter({1.0: 2,
         9.0: 2,
         23.0: 2,
         27.0: 2,
         36.0: 2,
         37.0: 2,
         38.0: 2,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         11.0: 1,
         12.0: 1,
         21.0: 1,
         39.0: 1,
         43.0: 1,
         48.0: 1})

In [58]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [8, 10, 11, 22, 28, 54, 62, 81, 83, 86, 90, 94, 97, 108, 129, 144, 153, 154, 158, 162, 173, 174, 176]
1: [8, 10, 11, 13, 16, 22, 28, 34, 47, 51, 54, 62, 73, 81, 83, 86, 90, 92, 94, 97, 98, 99, 108, 116, 129, 131, 132, 144, 150, 152, 153, 154, 158, 159, 161, 162, 166, 169, 173, 174, 176, 177, 178, 180, 182]
3: [1, 2, 8, 10, 11, 13, 15, 16, 17, 20, 22, 28, 34, 39, 47, 48, 49, 51, 53, 54, 58, 60, 62, 65, 68, 70, 71, 72, 73, 74, 77, 78, 79, 81, 83, 85, 86, 89, 90, 92, 93, 94, 95, 97, 98, 99, 107, 108, 112, 113, 114, 115, 116, 117, 119, 124, 129, 131, 132, 133, 144, 146, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 164, 166, 169, 172, 173, 174, 176, 177, 178, 180, 182, 183]
NA: [0, 7, 9, 18, 19, 21, 25, 30, 35, 38, 44, 76, 87, 104, 106, 111, 128, 139, 147, 165, 167, 175, 179]


In [59]:
BB.head()

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference
0,2013,108.0,296.0,46.0,84.0,14.0,0.0,17.0,51.0,0.0,...,0.0,6.0,Matt,Adams,Matthew James,245.0,75.0,30.0,10.0,11.0
1,2013,9.0,18.0,3.0,4.0,1.0,0.0,1.0,3.0,0.0,...,2.0,4.0,Ehire,Adrianza,Ehire Enrique,195.0,73.0,29.0,1.0,5.0
2,2013,25.0,72.0,10.0,19.0,4.0,0.0,2.0,9.0,1.0,...,1.0,6.0,Abraham,Almonte,Abraham,223.0,70.0,29.0,1.0,2.0
3,2013,97.0,334.0,34.0,94.0,11.0,0.0,6.0,45.0,6.0,...,5.0,10.0,Yonder,Alonso,Yonder,230.0,73.0,31.0,11.0,12.0
4,2013,152.0,626.0,64.0,177.0,31.0,2.0,5.0,52.0,35.0,...,1.0,17.0,Jose,Altuve,Jose Carlos,166.0,66.0,28.0,13.0,0.0


In [60]:
BB[BB.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [61]:
DD[DD.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [62]:
EE[EE.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [63]:
AA[AA.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [64]:
CC[CC.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference
